<a href="https://colab.research.google.com/github/GeorgeTelles/portfolio_optimization/blob/main/Otimiza%C3%A7%C3%A3o_Portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
  <img src="https://raw.githubusercontent.com/GeorgeTelles/georgetelles/f69531ec6b293b5148563588a764c010015d315e/logo_clara.png" alt="logo clara" width="300" style="display: inline-block; vertical-align: top; margin-right: 10px;">
  <img src="https://raw.githubusercontent.com/GeorgeTelles/georgetelles/f69531ec6b293b5148563588a764c010015d315e/logo_dark.png" alt="logo dark" width="300" style="display: inline-block; vertical-align: top;">
</div>


#Otimização de portfolio

**Este projeto visa otimizar carteiras de investimento utilizando a técnica de Monte Carlo. A ferramenta realiza milhares de simulações para identificar a fronteira eficiente do portfólio e determinar os pesos ideais com base em duas principais estratégias: Maximização do Índice de Sharpe e Minimização da Volatilidade.**

#Instalando e importando Bibliotecas e Modulos

In [ ]:
!pip install finquant

In [ ]:
from finquant.portfolio import build_portfolio
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd

**Ativos da Carteira**

In [ ]:
ticks = ["^BVSP", "VALE3.SA", "PETR4.SA", "WEGE3.SA", "ABEV3.SA", "HGLG11.SA"]

**Data para Analise**

In [ ]:
start_date = "2021-01-01"
end_date = "2024-08-01"

**Obtendo dados do Yahoo Finance**

In [ ]:
pf = build_portfolio(names=ticks, start_date=start_date, end_date=end_date, data_api="yfinance")
pf.data

**Retornos Acumulados**

In [ ]:
pf.comp_cumulative_returns()

In [ ]:
pf.comp_cumulative_returns().plot()

**Fazendo simulações para acha a Frontera Eficiente (melhor combinação de pesos para a carteira)**

10 mil iterações com o método de Monte Carlo

In [ ]:
opt_w, opt_res = pf.mc_optimisation(num_trials = 10000)

In [ ]:
figure(figsize=(8,8), dpi = 120)

pf.mc_plot_results()

pf.ef_plot_efrontier()

pf.ef_plot_optimal_portfolios()

pf.plot_stocks()

plt.savefig("efficientefrontier.png")

**Resultado das 2 Melhores formas de Compor a carteira**

In [ ]:
opt_w*100

In [ ]:
#resultados das carteiras
opt_res_2 = opt_res.copy()
opt_res_2["Expected Return"] = opt_res["Expected Return"]*100
opt_res_2["Volatility"] = opt_res["Volatility"]*100
opt_res_2


#Graficos de distribuição das Carteiras

In [ ]:
# Extraindo os pesos ideais para cada tipo de carteira
weights_min_volatility = opt_w.loc['Min Volatility']
weights_max_sharpe_ratio = opt_w.loc['Max Sharpe Ratio']

# Configuração da figura e eixos
fig, axs = plt.subplots(2, 1, figsize=(14, 14))

# Gráfico para a carteira com menor volatilidade
axs[0].pie(
    weights_min_volatility,
    labels=weights_min_volatility.index,
    autopct='%1.1f%%',
    colors=plt.cm.Paired(range(len(weights_min_volatility))),
    startangle=140,
    wedgeprops={'edgecolor': 'black'}
)
axs[0].set_title('Pesos Ideais - Menor Volatilidade')
axs[0].legend(
    loc='best',
    labels=[f'{label}: {weight*100:.2f}%' for label, weight in zip(weights_min_volatility.index, weights_min_volatility)],
    title='Ativos',
    bbox_to_anchor=(1, 1)  # Ajusta a posição da legenda
)

# Gráfico para a carteira com máximo Sharpe Ratio
axs[1].pie(
    weights_max_sharpe_ratio,
    labels=weights_max_sharpe_ratio.index,
    autopct='%1.1f%%',
    colors=plt.cm.Paired(range(len(weights_max_sharpe_ratio))),
    startangle=140,
    wedgeprops={'edgecolor': 'black'}
)
axs[1].set_title('Pesos Ideais - Máximo Sharpe Ratio')
axs[1].legend(
    loc='best',
    labels=[f'{label}: {weight*100:.2f}%' for label, weight in zip(weights_max_sharpe_ratio.index, weights_max_sharpe_ratio)],
    title='Ativos',
    bbox_to_anchor=(1, 1)
)

plt.tight_layout()
plt.show()